In [ ]:
from functions import *

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.windows import Window
import fiona
from shapely.geometry import shape

### Project leboiron linestrings into points

In [ ]:
import numpy as np
from rasterio.features import shapes

In [ ]:
import os
import fiona
import geopandas as gpd
from shapely.geometry import shape
import matplotlib.pyplot as plt
from shapely.ops import unary_union

rivers = {}

output_directory = '../data/external/shp/clipped_rivers_grid_cells_ogr'
a=0
for i,riv in enumerate(os.listdir(output_directory)):
    # Check if the file is a shapefile
    try:
        if riv.endswith('.shp'):
            a+=1
            print(f"Processing file: {riv}")
            river = gpd.read_file(os.path.join(output_directory, riv))
            combined_river = unary_union(river.geometry)
            
            total_length = combined_river.length  # Longitud total en metros

            # Calcular el área del bounding box de la geometría combinada
            bounding_box = river.total_bounds  # [minx, miny, maxx, maxy]
            width = bounding_box[2] - bounding_box[0]
            height = bounding_box[3] - bounding_box[1]
            area = width * height  # Área en metros 

            bounding_box_diagonal = ((bounding_box[2] - bounding_box[0])**2 + (bounding_box[3] - bounding_box[1])**2)**0.5

            length_percentage = (total_length / bounding_box_diagonal) * 100  # Comparación en función de la diagonal
            
            # Definir umbrales
            min_area_threshold = 225_000  # Área mínima en m² (0.225 km²)
            min_length_threshold = 300  # Longitud mínima en metros

            # Plot the GeoDataFrame
            #river = river.to_crs("EPSG:4326")
            print('Tinc area i longitud', area, total_length)
            #rivers[f'cell_{i}'] = river
            if area > min_area_threshold and total_length > min_length_threshold and length_percentage > 50:
                river.plot()
                plt.title(f"River data from: {riv}")
                plt.show()
                river = river.to_crs("EPSG:2056")
                rivers[f'cell_{i}'] = river

    except Exception as e:
        print(f"Error processing file: {riv}",e)
        pass
        

## Air interpolation

In [ ]:
rivers['cell_4']


In [ ]:
air_stations_total_metadata

In [ ]:
start_date = '2013-03-01'
end_date = '2019-12-31'

In [ ]:
raw_data_path = '../data/preprocessed/wt_interpolated'

os.makedirs(raw_data_path, exist_ok=True)
clear_directory(raw_data_path)

for r in rivers.keys():
  dest_dir = os.path.join(raw_data_path, r)
  os.makedirs(dest_dir, exist_ok=True)
  clear_directory(dest_dir)

In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm


def get_closest_grid_cell(x, y, x_range, y_range):
    x_idx = np.argmin(np.abs(x_range - x))
    y_idx = np.argmin(np.abs(y_range - y))
    return x_idx, y_idx

def generate_raster(df, shp, river, folder_path,var,date=None):

    # Extract the coordinate columns (Coord X, Coord Y)
    x_coords = df['Coord X'].values
    y_coords = df['Coord Y'].values
    
    # Define bounds and resolution for the grid
    bounds = shp.total_bounds
    resolution = 30
    x_min, y_min, x_max, y_max = bounds
    x_range = np.arange(x_min, x_max, resolution)
    y_range = np.arange(y_min, y_max, resolution)
    
    # Precompute the grid cell indices for all coordinates
    grid_indices = [get_closest_grid_cell(x, y, x_range, y_range) for x, y in zip(x_coords, y_coords)]
    
    # Define the transformation for the raster grid
    transform = from_origin(x_min, y_max, resolution, resolution)
    
    # Loop through each temperature column (each date)
    #for col in df.columns[2:]:  # Assuming temperature columns start from the 4th column onward

    #var = df[col].values  # Extract the variable data for the current date
    
    var_values = df[var].values

    # Initialize an array to store temperature values for the current day

    raster_array = np.ones((len(y_range), len(x_range)))

    # Assign temperature values to the grid cells based on precomputed indices
    for i, (x_idx, y_idx) in enumerate(grid_indices):
        temp = var_values[i]
        raster_array[y_idx, x_idx] = temp
        #print('li poso', temp)
    filename = f'{folder_path}/{date}.tif' if date != None else f'{folder_path}/{river}.tif'
    save_raster(np.flipud(raster_array),filename, shp)
    print('Saved at', filename)

    vmin = np.min(raster_array)
    vmax = np.max(raster_array)
    if vmin == vmax:
        vmin -= 0.01  # Slight adjustment to avoid identical vmin and vmax
    try:
        plt.imshow(raster_array, cmap='inferno', origin='lower')#, extent=[x_min, x_max, y_min, y_max],norm=LogNorm())
        plt.colorbar(label='1')
        plt.title(f'{var} Heatmap for {river}')
        plt.show()

    except ValueError as e:
        try:
            plt.imshow(raster_array, cmap='inferno', origin='lower', extent=[x_min, x_max, y_min, y_max],norm=LogNorm(vmin = vmin, vmax = vmax))
            plt.colorbar(label='2')
            plt.title(f'{var} Heatmap for {river}')
            plt.show()
        except:
            plt.imshow(raster_array, cmap='inferno', origin='lower', extent=[x_min, x_max, y_min, y_max])
            plt.colorbar(label='3')
            plt.title(f'{var} Heatmap for {river}')
            plt.show()


In [ ]:
DATA_PATH = '../data/external/airtemp_files'
for k,river in rivers.items():
    folder_path = f'../data/preprocessed/air_interpolated/{k}'
    os.makedirs(folder_path, exist_ok=True)
    clear_directory(folder_path)
    print('fent',k)
    x_coords, y_coords, z_coords = project_linestrings_to_points(river)
    #print(x_coords,y_coords)
    
    if z_coords == []:
        dir_altitudes = '../data/external/altitudes'
        with rasterio.open('../../../../../data/simon.walther/swissAltitude/swissAltitude.vrt') as src:
            for i in range(len(x_coords)):
                lon, lat = x_coords[i], y_coords[i]
                row, col = src.index(lon, lat)
                window = Window(col_off=col, row_off=row, width=1, height=1)
                pixel_value = src.read(1, window=window)
                z_coords.append(pixel_value[0][0])
                
    grid_x, grid_y = np.mgrid[min(x_coords):max(x_coords):256j, min(y_coords):max(y_coords):256j]

    # Interpolación de los valores de altitud en la cuadrícula
    grid_z = griddata((x_coords, y_coords), z_coords, (grid_x, grid_y), method='cubic')
    altitude_path = f'../data/preprocessed/altitude/{k}.tif'
    save_raster(grid_z, altitude_path, river)
    print('Saved', altitude_path)
        
    projection_coordenates = pd.DataFrame(data={'Coord X': x_coords,'Coord Y': y_coords,'Altitude': z_coords})
    
    air_stations_total_data = pd.read_csv(os.path.join(DATA_PATH, '30MinFreq_air_boiron_and_swissmeteo_data.csv'),index_col=0)
    air_stations_total_metadata = pd.read_csv(os.path.join(DATA_PATH, 'air_boiron_and_swissmeteo_metadata.csv'),index_col=0)
    
    # Filter the data to the selected time period
    air_stations_filt = air_stations_total_data.loc[start_date:end_date]
    air_stations_filt.index = pd.to_datetime(air_stations_filt.index)
    air_stations_monthly_mean = air_stations_filt.resample('M').mean()

    # Interpolate the data
    LR = 5.5
    air_interpolated = pd.DataFrame(columns=projection_coordenates.index)

    for ri,r in air_stations_monthly_mean.iterrows():
        temp = pd.concat([pd.DataFrame(r.values, index=r.index, columns=['temp',]), air_stations_total_metadata], axis=1)
        temp.dropna(inplace=True)

        temp['temp'] += LR * temp['Altitude'] / 1000

        if len(temp) == 0:
            a = np.empty(projection_coordenates.shape[0])
            a[:] = np.nan
            air_interpolated.loc[ri] = a
        else:
            x = temp['Coord X']
            y = temp['Coord Y']
            z = temp['temp']

            xi = projection_coordenates['Coord X']
            yi = projection_coordenates['Coord Y']

            temp_interpolated = simple_idw(x, y, z, xi, yi, beta=2)
            temp_interpolated -= LR * projection_coordenates['Altitude']  / 1000

            air_interpolated.loc[ri] = temp_interpolated
            
            df_air = pd.DataFrame(np.transpose(temp_interpolated))
            var = 'water_temp'
            df_air.columns = {var}
            #print('aaaaa')
            #print(df_air)
            #print(df_air.columns)
            df = pd.concat([projection_coordenates,df_air],axis =1,ignore_index = False)
            #print(df)
            
            generate_raster(df, river, k, folder_path, var, ri.strftime("%Y-%m"))
        
    #air_interpolated.to_csv(os.path.join(raw_data_path,'air_interpolated_leboiron.csv'), header=True)

In [ ]:
air_interpolated.to_csv(os.path.join(raw_data_path,'air_interpolated_leboiron.csv'), header=True)

#### Show and save results

In [ ]:
# Function to map coordinates to the grid (pre-compute grid indices)
def get_closest_grid_cell(x, y, x_range, y_range):
    x_idx = np.argmin(np.abs(x_range - x))
    y_idx = np.argmin(np.abs(y_range - y))
    return x_idx, y_idx

In [ ]:
folder_path = '../data/raw/airtemp_interpolated'

# List all files in the folder and delete them
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
    except Exception as e:
        print(f"Error: {e}")


In [ ]:
for k,river in rivers.items():
    
    df = projection_coordenates.join(np.transpose(air_interpolated))

    # Extract the coordinate columns (Coord X, Coord Y)
    x_coords = df['Coord X'].values
    y_coords = df['Coord Y'].values

    # Define bounds and resolution for the grid
    bounds = river.total_bounds
    resolution = 30
    x_min, y_min, x_max, y_max = bounds
    x_range = np.arange(x_min, x_max, resolution)
    y_range = np.arange(y_min, y_max, resolution)

    # Precompute the grid cell indices for all coordinates
    grid_indices = [get_closest_grid_cell(x, y, x_range, y_range) for x, y in zip(x_coords, y_coords)]

    # Define the transformation for the raster grid
    transform = from_origin(x_min, y_max, resolution, resolution)

    # Loop through each temperature column (each date)
    for col in df.columns[3:]:  # Assuming temperature columns start from the 4th column onward
        temperature = df[col].values  # Extract the temperature data for the current date

        # Initialize an array to store temperature values for the current day
        raster_array = np.zeros((len(y_range), len(x_range)))

        # Assign temperature values to the grid cells based on precomputed indices
        for i, (x_idx, y_idx) in enumerate(grid_indices):
            temp = temperature[i]
            raster_array[y_idx, x_idx] = temp

        save_raster(np.flipud(raster_array),f'{folder_path}/airtemp_{col.strftime("%Y-%m")}.tif',river)


        # Optional: Visualize the temperature grid for each day
        plt.imshow(raster_array, cmap='hot', origin='lower', extent=[x_min, x_max, y_min, y_max])
        plt.colorbar(label='Temperature')
        plt.title(f'Temperature Heatmap for {col}')
        plt.show()


## Water interpolation

#### Load data and filter

In [ ]:
water_stations = pd.read_csv('../data/external/watertemp files/water_boiron_2011_2023.csv')
water_stations

In [ ]:
start_date = pd.to_datetime('01-03-2013', dayfirst=True)  # Define start date in the correct format
end_date = pd.to_datetime('25-07-2023', dayfirst=True)    # You can set end date as needed

In [ ]:
# Asegúrate de que la columna de fechas esté en formato datetime
water_stations['Date et heure'] = pd.to_datetime(water_stations['Date et heure'],format='%d.%m.%Y %H:%M:%S', dayfirst=True)

# Establecer la columna 'Date et heure' como índice
water_stations.set_index('Date et heure', inplace=True)

# Filtrar por el rango de fechas deseado
water_stations_filt = water_stations.loc[start_date:end_date]

# Calcular la media mensual
water_stations_monthly_mean = water_stations_filt.resample('M').mean()


Load water stations metadata

In [ ]:
water_stations_total_metadata = pd.read_csv('../data/external/watertemp files/water_stations_with_Qratio.csv')
water_stations_total_metadata

#### Interpolation process

In [ ]:
water_stations_total_metadata.index = water_stations_total_metadata['CODE_MdlR']
water_stations_total_metadata

In [ ]:
import numpy as np
LR = 5.5
water_interpolated = pd.DataFrame(columns=projection_coordenates.index)

for ri,r in water_stations_monthly_mean.iterrows():
    temp = pd.concat([pd.DataFrame(r.values, index=r.index, columns=['temp',]),water_stations_total_metadata], axis=1)
    temp.dropna(inplace=True)

    temp['temp'] += LR * temp['Altitude'] / 1000
    if len(temp) == 0:
        a = np.empty(projection_coordenates.shape[0])
        a[:] = np.nan
        water_interpolated.loc[ri] = a
    else:
        x = temp['Coord X']
        y = temp['Coord Y']
        z = temp['temp']

        xi = projection_coordenates['Coord X']
        yi = projection_coordenates['Coord Y']

        temp_interpolated = simple_idw(x, y, z, xi, yi, beta=2)
        temp_interpolated -= LR * projection_coordenates['Altitude']  / 1000

        water_interpolated.loc[ri] = temp_interpolated
        print(temp_interpolated)

In [ ]:
water_interpolated.to_csv('../data/external/water_interpolated_leboiron.csv')

In [ ]:
df = projection_coordenates.join(np.transpose(water_interpolated))

#### Show and save results

In [ ]:
folder_path = '../data/raw/wt_interpolated'

# List all files in the folder and delete them
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
    except Exception as e:
        print(f"Error: {e}")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
import matplotlib.pyplot as plt

# Extract the coordinate columns (Coord X, Coord Y)
x_coords = df['Coord X'].values
y_coords = df['Coord Y'].values

# Define bounds and resolution for the grid
bounds = leboiron.total_bounds
resolution = 30
x_min, y_min, x_max, y_max = bounds
x_range = np.arange(x_min, x_max, resolution)
y_range = np.arange(y_min, y_max, resolution)

# Function to map coordinates to the grid (pre-compute grid indices)
def get_closest_grid_cell(x, y, x_range, y_range):
    x_idx = np.argmin(np.abs(x_range - x))
    y_idx = np.argmin(np.abs(y_range - y))
    return x_idx, y_idx

# Precompute the grid cell indices for all coordinates
grid_indices = [get_closest_grid_cell(x, y, x_range, y_range) for x, y in zip(x_coords, y_coords)]

# Define the transformation for the raster grid
transform = from_origin(x_min, y_max, resolution, resolution)

# Loop through each temperature column (each date)
for col in df.columns[3:]:  # Assuming temperature columns start from the 4th column onward
    temperature = df[col].values  # Extract the temperature data for the current date

    # Initialize an array to store temperature values for the current day
    raster_array = np.zeros((len(y_range), len(x_range)))

    # Assign temperature values to the grid cells based on precomputed indices
    for i, (x_idx, y_idx) in enumerate(grid_indices):
        temp = temperature[i]
        raster_array[y_idx, x_idx] = temp

    save_raster(np.flipud(raster_array),f'{folder_path}/watertemp_{col.strftime("%Y-%m")}.tif',leboiron)


    # Optional: Visualize the temperature grid for each day
    plt.imshow(raster_array, cmap='hot', origin='lower', extent=[x_min, x_max, y_min, y_max])
    plt.colorbar(label='Temperature')
    plt.title(f'Temperature Heatmap for {col}')
    plt.show()


## Slope and discharge interpolation

In [ ]:
print(len(rivers))

In [ ]:
file_path = '../data/external/slope_discharge/Typisierung_LV95/typisierung.gpkg'
gdf = gpd.read_file(file_path)

In [ ]:
#gdf = gdf.to_crs("EPSG:4326")

In [ ]:
print(gdf.crs)
print(rivers['LaVenoge_shapefile'].crs)

In [ ]:
gdf.plot()

In [ ]:
rivers['Malbunbach'].plot()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Graficar el primer GeoDataFrame (gdf1) en azul
gdf.plot(ax=ax, color='blue', edgecolor='black', label='GDF 1')

# Graficar el segundo GeoDataFrame (gdf2) en rojo
rivers['Malbunbach'].plot(ax=ax, color='red', edgecolor='black', label='GDF 2')

# Agregar leyenda
plt.legend()

# Mostrar el gráfico
plt.show()

In [ ]:
gdf[gdf.intersects(rivers['Malbunbach'].unary_union)]

In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm


def get_closest_grid_cell(x, y, x_range, y_range):
    x_idx = np.argmin(np.abs(x_range - x))
    y_idx = np.argmin(np.abs(y_range - y))
    return x_idx, y_idx

def generate_raster(df, shp, river, folder_path):
    print('holaaa')

    # Extract the coordinate columns (Coord X, Coord Y)
    x_coords = df['Coord X'].values
    y_coords = df['Coord Y'].values
    
    # Define bounds and resolution for the grid
    bounds = shp.total_bounds
    resolution = 30
    x_min, y_min, x_max, y_max = bounds
    x_range = np.arange(x_min, x_max, resolution)
    y_range = np.arange(y_min, y_max, resolution)
    
    # Precompute the grid cell indices for all coordinates
    grid_indices = [get_closest_grid_cell(x, y, x_range, y_range) for x, y in zip(x_coords, y_coords)]
    
    # Define the transformation for the raster grid
    transform = from_origin(x_min, y_max, resolution, resolution)
    
    # Loop through each temperature column (each date)
    for col in df.columns[2:]:  # Assuming temperature columns start from the 4th column onward
        print(col)
        var = df[col].values  # Extract the variable data for the current date
    
        # Initialize an array to store temperature values for the current day
        raster_array = np.zeros((len(y_range), len(x_range)))
    
        # Assign temperature values to the grid cells based on precomputed indices
        for i, (x_idx, y_idx) in enumerate(grid_indices):
            temp = var[i]
            raster_array[y_idx, x_idx] = temp
    
        save_raster(np.flipud(raster_array),f'{folder_path}/discharge_{river}.tif', shp)
    
        vmin = np.min(raster_array)
        vmax = np.max(raster_array)
        if vmin == vmax:
            vmin -= 0.01  # Slight adjustment to avoid identical vmin and vmax
        print('helou',vmin, vmax)
        try:
            plt.imshow(raster_array, cmap='inferno', origin='lower', extent=[x_min, x_max, y_min, y_max],norm=LogNorm())
            plt.colorbar(label='1')
            plt.title(f'{col} Heatmap for {river}')
            plt.show()

        except ValueError as e:
            try:
                plt.imshow(raster_array, cmap='inferno', origin='lower', extent=[x_min, x_max, y_min, y_max],norm=LogNorm(vmin = vmin, vmax = vmax))
                plt.colorbar(label='2')
                plt.title(f'{col} Heatmap for {river}')
                plt.show()
            except:
                plt.imshow(raster_array, cmap='inferno', origin='lower', extent=[x_min, x_max, y_min, y_max])
                plt.colorbar(label='3')
                plt.title(f'{col} Heatmap for {river}')
                plt.show()


In [ ]:
from scipy.interpolate import griddata
PATH_DATA = '../data/external/'
for k,river in rivers.items():

    print('fent',k)
    x_coords, y_coords, z_coords = project_linestrings_to_points(river)
    
    if z_coords == []:
        dir_altitudes = '../data/external/altitudes'
        with rasterio.open('../../../../../data/simon.walther/swissAltitude/swissAltitude.vrt') as src:
            for i in range(len(x_coords)):
                lon, lat = x_coords[i], y_coords[i]
                row, col = src.index(lon, lat)
                window = Window(col_off=col, row_off=row, width=1, height=1)
                pixel_value = src.read(1, window=window)
                z_coords.append(pixel_value[0][0])
    
    # Crear una cuadrícula (grid) de puntos donde interpolar los valores
    grid_x, grid_y = np.mgrid[min(x_coords):max(x_coords):256j, min(y_coords):max(y_coords):256j]

    # Interpolación de los valores de altitud en la cuadrícula
    grid_z = griddata((x_coords, y_coords), z_coords, (grid_x, grid_y), method='cubic')

    # Visualizar la imagen resultante con matplotlib
    plt.imshow(grid_z, extent=(min(x_coords), max(x_coords), min(y_coords), max(y_coords)), origin='lower', cmap='terrain')
    plt.colorbar(label='Altitud')
    plt.title('Altitud Interpolada')
    plt.show()

    projection_coordenates = pd.DataFrame(data={'Coord X': x_coords,'Coord Y': y_coords})#,'Altitude': z_coords})
    print(projection_coordenates.shape)
    print('done coordenates')
    
    gdf_filtered = gdf[gdf.intersects(river.unary_union)]
    if not gdf_filtered.empty:
        disch = []
        slopes = []
        x_coords_gdf = []
        y_coords_gdf = []
        z_coords_gdf = []
        for i,row in gdf_filtered.iterrows():
            geom = row.geometry
            for linestring in geom.geoms:
                for coord in linestring.coords:
                    if k == 'Malbunbach':
                        print('Coordenades:', coord[0], coord[1])
                    x_coords_gdf.append(coord[0])
                    y_coords_gdf.append(coord[1])
                    if len(coord) == 3:
                        z_coords_gdf.append(coord[2])
                    disch.append(row.Discharge)
                    slopes.append(row.Slope)
    
        
        print(len(x_coords_gdf), len(y_coords_gdf),len(disch), len(slopes))           
    
        print('done gdf coordenates')
        projection_coordenates_filtered = pd.DataFrame(data={'Discharge': disch,'Slope':slopes,'Coord X': x_coords_gdf,'Coord Y': y_coords_gdf})#,'Altitude': z_coords_gdf})
        print(projection_coordenates_filtered.shape)
        
        # Interpolate the data
        LR = 5.5
        discharge_interpolated_df = pd.DataFrame(columns=projection_coordenates.index)
        slope_interpolated_df = pd.DataFrame(columns=projection_coordenates.index)
        
    
        temp = projection_coordenates_filtered.copy()
        #temp.dropna(inplace=True)
    
        if len(temp) == 0:
            a = np.empty(projection_coordenates.shape[0])
            a[:] = np.nan
            discharge_interpolated = a
        else:
            x = temp['Coord X']
            y = temp['Coord Y']
            z = temp['Discharge']
            z_slope = temp['Slope']
            
    
            xi = projection_coordenates['Coord X']
            yi = projection_coordenates['Coord Y']
            
            #print('Soc discharge abans',z)
            discharge_interpolated = simple_idw(x, y, z, xi, yi, beta=2)
            #print('Soc discharge',discharge_interpolated)
            #temp_interpolated -= LR * projection_coordenates['Altitude']  / 1000
            discharge_interpolated_df.loc[0]=discharge_interpolated
    
            slope_interpolated = simple_idw(x, y, z_slope, xi, yi, beta=2)
            slope_interpolated_df.loc[0]=slope_interpolated
    
        #discharge_interpolated_df.to_csv(os.path.join(PATH_DATA,f'discharge/discharge_interpolated_{k}.csv'), header=True)
        #slope_interpolated_df.to_csv(os.path.join(PATH_DATA,f'slope/slope_interpolated_{k}.csv'), header=True)
        
        df_disch = np.transpose(discharge_interpolated_df)
        df_disch.columns = {'Discharge'}
        df = pd.concat([projection_coordenates,df_disch],axis =1,ignore_index = False)
    
        folder_path = f'../data/preprocessed/discharge/{k}'
        os.makedirs(folder_path, exist_ok=True)
        clear_directory(folder_path)
        generate_raster(df, river, k, folder_path)
        
        df_slope = np.transpose(slope_interpolated_df)
        df_slope.columns = {'Slope'}
        df = pd.concat([projection_coordenates,df_slope],axis =1,ignore_index = False)
        
        folder_path = f'../data/preprocessed/slope/{k}'
        os.makedirs(folder_path, exist_ok=True)
        clear_directory(folder_path)
        generate_raster(df, river, k, folder_path)
        
        
        print('done',k)

## Water dataset interpolation

In [ ]:
gdf_points = gpd.read_file('../data/external/wt_filtered/wt_filtered.geojson')

In [ ]:
import numpy as np
LR = 5.5

for k, river in rivers.items():
    x_coords, y_coords, z_coords = project_linestrings_to_points(river)
    
    if z_coords == []:
        dir_altitudes = '../data/external/altitudes'
        with rasterio.open('../../../../../data/simon.walther/swissAltitude/swissAltitude.vrt') as src:
            for i in range(len(x_coords)):
                lon, lat = x_coords[i], y_coords[i]
                row, col = src.index(lon, lat)
                window = Window(col_off=col, row_off=row, width=1, height=1)
                pixel_value = src.read(1, window=window)
                z_coords.append(pixel_value[0][0])
    
    projection_coordenates = pd.DataFrame(data={'Coord X': x_coords,'Coord Y': y_coords,'Altitude': z_coords})
    
    water_coordinates = pd.DataFrame(data = {'Coord X': gdf_points['latitude'],'Coord Y': gdf_points['longitude'],'watertemp':gdf_points['waterTemperature']})
    
    water_interpolated = pd.DataFrame(columns=projection_coordenates.index)
    temp = water_coordinates.copy()

    if len(temp) == 0:
        a = np.empty(projection_coordenates.shape[0])
        a[:] = np.nan
        discharge_interpolated = a
    else:
        x = temp['Coord X']
        y = temp['Coord Y']
        z = temp['watertemp']

        xi = projection_coordenates['Coord X']
        yi = projection_coordenates['Coord Y']
        
        #print('Soc discharge abans',z)
        w_interpolated = simple_idw(x, y, z, xi, yi, beta=2)
        #print('Soc discharge',discharge_interpolated)
        #temp_interpolated -= LR * projection_coordenates['Altitude']  / 1000
        water_interpolated.loc[0]=w_interpolated

    df_disch = np.transpose(discharge_interpolated_df)
    df_disch.columns = {'Discharge'}
    df = pd.concat([projection_coordenates,df_disch],axis =1,ignore_index = False)

    folder_path = f'../data/preprocessed/wt_interpolated/{k}'
    os.makedirs(folder_path, exist_ok=True)
    clear_directory(folder_path)
    generate_raster(df, river, k, folder_path)
    print('Done',k)
